In [16]:
from common_immunogit import *

In [12]:
# TEMPORARY : 
from matplotlib_venn import venn3

""" Basic statistics and plots: Venn Diagram, etc. """

# Load the data from the JSON file
file_path = str(root_path / "metadata/model_metadata_tags.json")
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Initialize tag counts
tag_counts = defaultdict(int)

# Count tags
for model_info in data.values():
    if "tags" in model_info:
        for tag in model_info["tags"]:
            if ":" in tag:
                _, term = map(str.strip, tag.split(":", 1))
                tag_counts[term.lower()] += 1

# Keywords dictionary
keywords_file = root_path / "tmp/keywords.json"
with open(keywords_file, "r", encoding="utf-8") as f:
    keywords_dict = json.load(f)

# Define categories and initialize sets for Venn diagram
categories = list(keywords_dict.keys())
venn_sets = {cat: set() for cat in categories}
occurrences = {cat: defaultdict(int) for cat in categories}

# Populate Venn sets and occurrences
for category in categories:
    for term in keywords_dict[category]:
        term_lower = term.lower().strip()
        if term_lower in tag_counts:
            venn_sets[category].add(term)
            occurrences[category][term] = tag_counts[term_lower]

# Plot Venn diagram
plt.figure(figsize=(6, 6))
venn = venn3([venn_sets.get("Immunology", set()), 
              venn_sets.get("Treatments", set()), 
              venn_sets.get("Cells", set())],
             set_labels=categories)

# Add occurrences to Venn diagram
for category, idx in zip(categories, range(3)):
    for term in venn_sets[category]:
        subset = venn.subset_labels[idx]
        if subset:
            label = f'{term}\n({occurrences[category].get(term, 0)})'
            subset.set_text(label)

# Adjust font size and add title
for subset in venn.subset_labels:
    if subset:
        subset.set_fontsize(10)

plt.title("Venn Diagram")
plt.show()

ModuleNotFoundError: No module named 'matplotlib_venn'

In [13]:
""" Basic Dataframe for Metrics """
file_path = "model_metadata_tags.json"
with open(file_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# for the titles
title_keywords = ["immune system", "immune response", "therapy"]

tag_counts = defaultdict(int)

title_classification = {}

for model_id, model_info in data.items():
    title = model_info.get("title", "").lower()
    title_category = "OTHER"

    for keyword in title_keywords:
        if keyword in title:
            title_category = keyword
            break

    title_classification[model_id] = title_category

    if "tags" in model_info:
        for tag in model_info["tags"]:
            if ":" in tag:
                _, term = map(str.strip, tag.split(":", 1))
                tag_counts[term.lower()] += 1  # Convertir en minuscules pour une comparaison insensible à la casse

count_table = []

for category, terms in keywords_dict.items():
    for term in terms:
        term_lower = term.lower().strip() # lowercase, no spacing !
        count = tag_counts.get(term_lower, 0)
        count_table.append({"Category": category, "Term": term, "Count": count})

# classify per titles
count_table.append({"Category": "Title", "Term": "immune system", "Count": list(title_classification.values()).count("immune system")})
count_table.append({"Category": "Title", "Term": "immune response", "Count": list(title_classification.values()).count("immune response")})
count_table.append({"Category": "Title", "Term": "therapy", "Count": list(title_classification.values()).count("therapy")})
count_table.append({"Category": "Title", "Term": "OTHER", "Count": list(title_classification.values()).count("OTHER")})

df_counts = pd.DataFrame(count_table)
df_counts.to_csv("model_metadata_counts.tsv", sep='\t', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'model_metadata_tags.json'

In [14]:
df_counts

NameError: name 'df_counts' is not defined